# Visualising PAG neurons in CCF space
In this notebook we will load the .csv file containing the metadata from our PAG_scRNAseq project and use the CCF coordinates obtained after registration with Sharp-Track to visualise our sequenced cells with Brainrender.

### 1 | Import packages and set defaults

In [ ]:
import brainrender
from brainrender import Scene, Animation
from brainrender.actors import Points
from vedo import settings as vsettings
from brainrender.video import VideoMaker
import pandas as pd # used to load the cwv
from vedo import embedWindow, Plotter, show  # <- this will be used to render an embedded scene 


# // DEFAULT SETTINGS //
# You can see all the default settings here: https://github.com/brainglobe/brainrender/blob/19c63b97a34336898871d66fb24484e8a55d4fa7/brainrender/settings.py

# --------------------------- brainrender settings --------------------------- #
# Change some of the default settings
brainrender.settings.BACKGROUND_COLOR = "white" # color of the background window (defaults to "white", try "blackboard")
brainrender.settings.DEFAULT_ATLAS = "allen_mouse_25um"  # default atlas
brainrender.settings.DEFAULT_CAMERA = "three_quarters"  # Default camera settings (orientation etc. see brainrender.camera.py)
brainrender.settings.INTERACTIVE = False  # rendering interactive ?
brainrender.settings.LW = 2  # e.g. for silhouettes
brainrender.settings.ROOT_COLOR = [0.4, 0.4, 0.4]  # color of the overall brain model's actor (defaults to [0.8, 0.8, 0.8])
brainrender.settings.ROOT_ALPHA = 0.2  # transparency of the overall brain model's actor (defaults to 0.2)
brainrender.settings.SCREENSHOT_SCALE = 1  # values >1 yield higher resolution screenshots
brainrender.settings.SHADER_STYLE = "cartoon"  # affects the look of rendered brain regions, values can be: ["metallic", "plastic", "shiny", "glossy", "cartoon"] and can be changed in interactive mode
brainrender.settings.SHOW_AXES = False
brainrender.settings.WHOLE_SCREEN = True  # If true render window is full screen
brainrender.settings.OFFSCREEN = False

# ------------------------------- vedo settings ------------------------------ #
# For transparent background with screenshots
vsettings.screenshotTransparentBackground = True  # vedo for transparent bg
vsettings.useFXAA = False  # This needs to be false for transparent bg


# // SET PARAMETERS //
# Save folder
save_folder = r"D:\Dropbox (UCL)\Project_transcriptomics\analysis\PAG_scRNAseq_brainrender\output"

#### 1.1 | Check atlas availability
Brainrender integrates several atlases that can be used to visualise and explore brain anatomy. We can check which atlases are available, take a look at the ones we have already downloaded, and render a basic scene with the axis to get an idea of the overall picture.

In [ ]:
# Run this to get a list of the available atlases:
from bg_atlasapi import show_atlases
show_atlases()

In [ ]:
# Find the dimensions of an atlas:
from bg_atlasapi.bg_atlas import BrainGlobeAtlas
atlas = BrainGlobeAtlas("allen_mouse_25um")
reference_image = atlas.reference
print(reference_image.shape)

Currently, among the atlases we can use for mouse data:
* allen_mouse_10um_v1.2 - Volume dimension of \[AP, SI, LR\] equivalent to \[1320, 800, 1140\]
* allen_mouse_25um_v1.2 - Volume dimension of \[AP, SI, LR\] equivalent to \[528, 320, 456\] (default atlas)
* kim_mouse_10um_v1.0 - Volume dimension of \[AP, SI, LR\] equivalent to \[1320, 800, 1140\]
* kim_unified_25um_v1.0 - Volume dimension of \[AP, SI, LR\] equivalent to \[528, 320, 456\]
* kim_unified_50um_v1.0 - Volume dimension of \[AP, SI, LR\] equivalent to \[264, 160, 228\]
* osten_mouse_10um_v1.1 - Volume dimension of \[AP, SI, LR\] equivalent to \[1320, 800, 1140\]

The CCF coordinates we obtained from Sharp-Track are at 10um resolution, with a Volume dimension of \[AP, SI, LR\] equivalent to \[1320, 800, 1140\]

In [ ]:
embedWindow(None)  # <- this will make your scene popup

# Create a scene with the with the preferred atlas and check the dimensions
brainrender.settings.SHOW_AXES = True
scene = Scene(root = True, atlas_name = 'allen_mouse_25um', inset = False, title = 'allen_mouse_25um', screenshots_folder = save_folder, plotter = None)

scene.render(interactive = True, camera = "sagittal", zoom = 1) # make sure to press 'Esc' to close not 'q' or kernel dies

### 2 | Load metadata including CCF coordinates
Other options to load registered points include `add_cells_from_file` and `get_probe_points_from_sharptrack`.

In [ ]:
# Load data
pag_data = pd.read_csv("D:\\Dropbox (UCL - SWC)\\Project_transcriptomics\\analysis\\PAG_scRNAseq_brainrender\\PAG_scRNAseq_metadata_211018.csv")

# Look at the first 5 rows of the metadata
pag_data.head()

In [ ]:
# List all column names in data
pag_data.columns

#### 2.1 | Scaling up coordinates
The CCF coordinates were obtained by registering images to the Allen Brain Atlas using Sharp-Track (see Shamash et al. bioRxiv 2018 and https://github.com/cortex-lab/allenCCF), which yields coordinates at a resolution of 10 micrometers. In the Allen Brain Atlas, a point at coordinates \[1, 0, 0\] is at 10um from the origin (in other words, 1 unit of the atlas space equals 10um). However, BrainRender's space is at 1um resolution, so the first thing we need to do is to scale up the coordinate values by 10 to get them to match correctly.

In [ ]:
# Inspect the CCF coordinates for each cell
pag_data[["cell.id", "cell.type", "PAG.area", "CCF.AllenAP", "CCF.AllenDV", "CCF.AllenML"]]

In [ ]:
# Scale up data
sharptrack_to_brainrender_scale_factor = 10 # Sharp-Track uses a 10um reference atlas so the coordinates need to be scaled to match brainrender's

pag_data["CCF.AllenAP"] *= sharptrack_to_brainrender_scale_factor
pag_data["CCF.AllenDV"] *= sharptrack_to_brainrender_scale_factor
pag_data["CCF.AllenML"] *= sharptrack_to_brainrender_scale_factor

pag_data[["cell.id", "cell.type", "CCF.AllenAP", "CCF.AllenDV", "CCF.AllenML"]]

### 3 | Rendering cells with Brainrender
Now that we have the coordinates at the right scale, we can render our cells in Brainrender and colour them according to any metadata we want. We will prepare different renderings in each of the following code chunks.

#### 3.1 | Selecting a subset of cells
We can first take a look at how to use the metadata to select subsets of cells. This will be useful to either render just some of the cells, or to color cells based on some metadata info.

In [ ]:
# We can subset cells using any criteria we want. For instance, let's keep cells coming from each hemisphere in separate variables:
cells_hemisphere_right = pag_data.loc[pag_data["PAG.hemisphere"] == "right"]
cells_hemisphere_left = pag_data.loc[pag_data["PAG.hemisphere"] == "left"]
cells_hemisphere_right.head()

In [ ]:
# We can also use multiple criteria at the same time, such as hemisphere and cell type:
vgat_cells_hemisphere_left = pag_data.loc[(pag_data["PAG.hemisphere"] == "left")&(pag_data["cell.type"] == "VGAT")]
vgat_cells_hemisphere_left.head()

We can now render a scene adding each subset independently, so we can tweak variables such as color or size separately. However, brainrender requires an array of 3 values as coordinates, so we need to get the values out instead of subsetting the dataframe and providing it as input when creating a scene: 

In [ ]:
column_names = ["CCF.AllenAP", "CCF.AllenDV", "CCF.AllenML"] # name of the columns containing the CCF coordinates
cells_hemisphere_right[column_names].head().values # brainrender needs a numpy array as coordinates. Without the .values we get a dataframe.

Now we can render a scene and color the cells on the right and left hemisphere differently:

In [ ]:
embedWindow(None)  # <- this will make your scene popup
brainrender.settings.SHOW_AXES = False # Set this back to False

# Create a variable containing the XYZ coordinates of the cells.
column_names = ["CCF.AllenAP", "CCF.AllenDV", "CCF.AllenML"] # name of the columns containing the CCF coordinates

# // CREATE SCENE //
scene = Scene(root = True, atlas_name = 'allen_mouse_25um', inset = False, title = 'Aspirated Cells', screenshots_folder = save_folder, plotter = None)

# // ADD REGIONS AND CELLS//
scene.add_brain_region("PAG", alpha = 0.1, color = "darkgoldenrod", silhouette = None, hemisphere = "both")
scene.add(Points(cells_hemisphere_right[column_names].values, name = "right hemisphere", colors = "salmon", alpha = 1, radius = 20, res = 16))
scene.add(Points(cells_hemisphere_left[column_names].values, name = "left hemisphere", colors = "skyblue", alpha = 1, radius = 20, res = 16))

# // RENDER INTERACTIVELY //
# Render interactively. You can press "s" to take a screenshot
scene.render(interactive = True, camera = "sagittal", zoom = 1) # make sure to press 'Esc' to close not 'q' or kernel dies

#### 3.2 | Rendering VGAT and VGluT2 cells
We can follow the strategy above and assign each cell type to a different variable and render them separately.

In [ ]:
embedWindow(None)  # <- this will make your scene popup
brainrender.settings.SHOW_AXES = True # Set this back to False
brainrender.settings.SCREENSHOT_SCALE = 10 # values >1 yield higher resolution screenshots
brainrender.settings.SHADER_STYLE = "plastic" # affects the look of rendered brain regions, values can be: ["metallic", "plastic", "shiny", "glossy", "cartoon"] and can be changed in interactive mode
brainrender.settings.ROOT_COLOR = [0.4, 0.4, 0.4] # color of the overall brain model's actor (defaults to [0.8, 0.8, 0.8])
brainrender.settings.ROOT_ALPHA = 0.2 # transparency of the overall brain model's actor (defaults to 0.2)
save_folder_thesis = r"D:\Dropbox (UCL)\Project_transcriptomics\analysis\PAG_scRNAseq_brainrender\output\figures_thesis_chapter_2"

# Create a variable containing the XYZ coordinates of the cells.
column_names = ["CCF.AllenAP", "CCF.AllenDV", "CCF.AllenML"] # name of the columns containing the CCF coordinates

# Color cells according to whether they are excitatory or inhibitory:
vgat_cells = pag_data.loc[pag_data["cell.type"] == "VGAT"]
vglut2_cells =  pag_data.loc[pag_data["cell.type"] == "VGluT2"]

# // CREATE SCENE //
scene = Scene(root = True, atlas_name = 'allen_mouse_25um', inset = False, title = None, screenshots_folder = save_folder_thesis, plotter = None)

# // ADD REGIONS AND CELLS//
scene.add_brain_region("PAG", alpha = 0.2, color = "darkgoldenrod", silhouette = None, hemisphere = "both")
#scene.add_brain_region("SCm", alpha = 0.1, color = "olivedrab", silhouette = None, hemisphere = "both")
scene.add(Points(vgat_cells[column_names].values, name = "vgat", colors = "#FF8080", alpha = 1, radius = 30, res = 16)) # colour is #FF8080 in figures, but salmon is the same
scene.add(Points(vglut2_cells[column_names].values, name = "vglut2", colors = "#0F99B2", alpha = 1, radius = 30, res = 16)) # colour is #0F99B2 in figures, but skyblue looks good here

# // RENDER INTERACTIVELY //
# Render interactively. You can press "s" to take a screenshot
scene.render(interactive = True, camera = "top", zoom = 1.1) # cameras can be "sagittal", "sagittal2", "frontal", "top", "top_side", "three_quarters"

#### 3.3 | Rendering cells according to PAG subdivision
Here, we render all the cells and color them with the PAG subdivision they were acquired from.

In [ ]:
embedWindow(None)  # <- this will make your scene popup
brainrender.settings.SHOW_AXES = False # Set this back to False

# Create a variable containing the XYZ coordinates of the cells.
column_names = ["CCF.AllenAP", "CCF.AllenDV", "CCF.AllenML"] # name of the columns containing the CCF coordinates

# Color cells according to their subdivision:
dmpag_cells = pag_data.loc[pag_data["PAG.area"] == "dmpag"]
dlpag_cells = pag_data.loc[pag_data["PAG.area"] == "dlpag"]
lpag_cells = pag_data.loc[pag_data["PAG.area"] == "lpag"]
vlpag_cells = pag_data.loc[pag_data["PAG.area"] == "vlpag"]

# // CREATE SCENE //
scene = Scene(root = True, atlas_name = 'allen_mouse_25um', inset = False, title = 'Aspirated Cells', screenshots_folder = save_folder, plotter = None)

# // ADD REGIONS AND CELLS//
scene.add_brain_region("PAG", alpha = 0.1, color = "darkgoldenrod", silhouette = None, hemisphere = "both")
scene.add_brain_region("SCm", alpha = 0.1, color = "olivedrab", silhouette = None, hemisphere = "both")
scene.add(Points(dmpag_cells[column_names].values, name = "dmpag", colors = "cornflowerblue", alpha = 1, radius = 25, res = 16))
scene.add(Points(dlpag_cells[column_names].values, name = "dlpag", colors = "darkorange", alpha = 1, radius = 25, res = 16))
scene.add(Points(lpag_cells[column_names].values, name = "lpag", colors = "forestgreen", alpha = 1, radius = 25, res = 16))
scene.add(Points(vlpag_cells[column_names].values, name = "vlpag", colors = "firebrick", alpha = 1, radius = 25, res = 16))

# // RENDER INTERACTIVELY //
# Render interactively. You can press "s" to take a screenshot
scene.render(interactive = True, camera = "three_quarters", zoom = 1) # choose one of the cameras: sagittal, sagittal2, frontal, top, top_side, three_quarters

#### 3.4 | Rendering cells according to cluster ID
Here, we render all the cells and color them with the cluster they were assigned to during analysis. 

In [ ]:
embedWindow(None)  # <- this will make your scene popup
brainrender.settings.SHOW_AXES = False # Set this back to False

# Create a variable containing the XYZ coordinates of the cells.
column_names = ["CCF.AllenAP", "CCF.AllenDV", "CCF.AllenML"] # name of the columns containing the CCF coordinates

# Color cells according to their subdivision:
cluster_0 = pag_data.loc[pag_data["SNN_clusters_cv2_jaccard_k8_letters"] == "zero"]
cluster_1 = pag_data.loc[pag_data["SNN_clusters_cv2_jaccard_k8_letters"] == "one"]
cluster_2 = pag_data.loc[pag_data["SNN_clusters_cv2_jaccard_k8_letters"] == "two"]
cluster_3 = pag_data.loc[pag_data["SNN_clusters_cv2_jaccard_k8_letters"] == "three"]
cluster_4 = pag_data.loc[pag_data["SNN_clusters_cv2_jaccard_k8_letters"] == "four"]
cluster_5 = pag_data.loc[pag_data["SNN_clusters_cv2_jaccard_k8_letters"] == "five"]
cluster_6 = pag_data.loc[pag_data["SNN_clusters_cv2_jaccard_k8_letters"] == "six"]
cluster_7 = pag_data.loc[pag_data["SNN_clusters_cv2_jaccard_k8_letters"] == "seven"]
cluster_8 = pag_data.loc[pag_data["SNN_clusters_cv2_jaccard_k8_letters"] == "eight"]
cluster_9 = pag_data.loc[pag_data["SNN_clusters_cv2_jaccard_k8_letters"] == "nine"]
cluster_10 = pag_data.loc[pag_data["SNN_clusters_cv2_jaccard_k8_letters"] == "ten"]
cluster_11 = pag_data.loc[pag_data["SNN_clusters_cv2_jaccard_k8_letters"] == "eleven"]

# // CREATE SCENE //
scene = Scene(root = True, atlas_name = 'allen_mouse_10um', inset = False, title = 'Aspirated Cells', screenshots_folder = save_folder, plotter = None)

# // ADD REGIONS AND CELLS//
scene.add_brain_region("PAG", alpha = 0.1, color = "darkgoldenrod", silhouette = None, hemisphere = "both")
scene.add_brain_region("SCm", alpha = 0.1, color = "olivedrab", silhouette = None, hemisphere = "both")
scene.add(Points(cluster_0[column_names].values, name = "cluster_0", colors = "lightgray", alpha = 1, radius = 20, res = 16))
scene.add(Points(cluster_1[column_names].values, name = "cluster_1", colors = "firebrick", alpha = 1, radius = 20, res = 16))
scene.add(Points(cluster_2[column_names].values, name = "cluster_2", colors = "navy", alpha = 1, radius = 20, res = 16))
scene.add(Points(cluster_3[column_names].values, name = "cluster_3", colors = "firebrick", alpha = 1, radius = 20, res = 16))
scene.add(Points(cluster_4[column_names].values, name = "cluster_4", colors = "navy", alpha = 1, radius = 20, res = 16))
scene.add(Points(cluster_5[column_names].values, name = "cluster_5", colors = "darkorange", alpha = 1, radius = 20, res = 16))
scene.add(Points(cluster_6[column_names].values, name = "cluster_6", colors = "powderblue", alpha = 1, radius = 20, res = 16))
scene.add(Points(cluster_7[column_names].values, name = "cluster_7", colors = "navy", alpha = 1, radius = 20, res = 16))
scene.add(Points(cluster_8[column_names].values, name = "cluster_8", colors = "navy", alpha = 1, radius = 20, res = 16))
scene.add(Points(cluster_9[column_names].values, name = "cluster_9", colors = "darkorange", alpha = 1, radius = 20, res = 16))
scene.add(Points(cluster_10[column_names].values, name = "cluster_10", colors = "powderblue", alpha = 1, radius = 20, res = 16))
scene.add(Points(cluster_11[column_names].values, name = "cluster_11", colors = "lightgray", alpha = 1, radius = 20, res = 16))

# // RENDER INTERACTIVELY //
# Render interactively. You can press "s" to take a screenshot
scene.render(interactive = True, camera = "three_quarters", zoom = 1) # choose one of the cameras: sagittal, sagittal2, frontal, top, top_side, three_quarters

# TO UPDATE
We can use this to manually curate the location of each cells after visual inspection. We can further curate the PAG subdivision by following to the next section, where we used the `kim_unified_25um_v0.1` atlas to extract the brain area from the coordinates. We will go over all the possibilities and then pool the results from the different approaches to populate a curated column in the metadata, which will be used for final plotting and further scRNA-seq data analysis.

### 4 | Using the enhanced Franklin-Paxinos atlas (Chon et al. 2019) to obtain PAG subdivisions from CCF coordinates
Another useful functionality of Brainrender is using `atlas.structure_from_coords()` to obtain the PAG subdivision each cell belongs to from the registered coordinates.

#### 4.1 | Using kim_unified_25um_v0.1

In [ ]:
# Scale up data: sharptrack uses a 10um reference atlas, brainrender's default is 1um, and the kim atlas is either 50um or 25um, so the coordinates need to be scaled
brainrender_to_kim_25_scale_factor = 0.04 # the kim atlas we are using is at 25um resolution so the coordinates need to be scaled to match the root

pag_data["CCF.AllenAP"] *= brainrender_to_kim_25_scale_factor
pag_data["CCF.AllenDV"] *= brainrender_to_kim_25_scale_factor
pag_data["CCF.AllenML"] *= brainrender_to_kim_25_scale_factor

pag_data[["cell.id", "cell.type", "CCF.AllenAP", "CCF.AllenDV", "CCF.AllenML"]]

In [ ]:
brainrender.settings.SHOW_AXES = False # Set this back to False

# Create a variable containing the XYZ coordinates of the cells.
column_names = ["CCF.AllenAP", "CCF.AllenDV", "CCF.AllenML"] # name of the columns containing the CCF coordinates

# To use the atlas with PAG subdivisions, try:
# kim_unified_50um_v0.1
# kim_unified_25um_v0.1

# Let's color cells according to whether they are excitatory or inhibitory:
vgat_cells = pag_data.loc[pag_data["cell.type"] == "VGAT"]
vglut2_cells =  pag_data.loc[pag_data["cell.type"] == "VGluT2"]

# // CREATE SCENE //
# Create a scene with no title. You can also use scene.add_text to add other text elsewhere in the scene
scene = Scene(display_inset = True, title = None, camera = "sagittal", screenshot_kwargs = screenshot_params, atlas = "kim_unified_25um_v0.1")
# Set add_root = False to focus only on the desired brain area

# // ADD CELLS//
scene.add_cells(vgat_cells, color = "salmon", color_by_region = False, col_names = column_names, radius = 1, res = 24, alpha = 1)
scene.add_cells(vglut2_cells, color = "skyblue", color_by_region = False, col_names = column_names, radius = 1, res = 24, alpha = 1)

# // ADD REGIONS //
scene.add_brain_regions(["PAG"],
    colors = get_random_colors(1), use_original_color = False)
# scene.add_brain_regions(["DMPAG", "DLPAG", "LPAG", 
# Try this:AG"],
#     color = dict(DLPAG = "blackboard", DMPAG = "olivedrab", LPAG = s"li[PAG = "red"), "nal_color = 
# // RENDER IVELY //
# Render interactively. You

# Or this: # pag_dict = dict(DLPAG = "blackboard", DMPAG = "olivedrab", LPAG = "lightgreen", VLPAG = "red")
# scene.add_brain_regions(pag_dict.keys(),
#     colors = pag_dict.values(), use_original_color = False)
can press "s" to take a screenshot
scene.render(interactive = True, video = False, camera = "sagittal", zoom = 1) # press 'Esc' to close not 'q' or kernel dies

In [ ]:
pag_data.head()
pag_data["brainrender.area"] = "empty"
pag_data.head()

In [ ]:
cell_region = scene.atlas.structure_from_coords([94.0, 397.2, 219.6], as_acronym = True)
cell_region
#pag_data.head()

In [ ]:
# Try the following to get the PAG area from the coordinates.
# [AP, SI, LR] = [1320, 800, 1140] --> 10um resolution (sharp-track)
# [AP, SI, LR] = [528, 320, 456] --> 25um resolution (kim_unified)
# in scene.atlas.structure_from_coords axis 0 is SI, 1 is AP, and 2 is LR, so [SI, AP, LR]

# cell_region = scene.atlas.structure_from_coords([94.0, 397.2, 219.6], as_acronym = True)
# cell_region

# coordinates = pag_data[["CCF.AllenAP", "CCF.AllenDV", "CCF.AllenML"]] # Original order
# coordinates = pag_data[["CCF.AllenDV", "CCF.AllenAP", "CCF.AllenML"]]

pag_data["brainrender.area"] = "empty"

for index, row in pag_data.iterrows():
    #print(scene.atlas.structure_from_coords([row["CCF.AllenDV"], row["CCF.AllenAP"], row["CCF.AllenML"]], as_acronym = True))
    #print(pag_data[row["cell.id"]])
    #pag_data[row["brainrender.area"]] = scene.atlas.structure_from_coords([row["CCF.AllenDV"], row["CCF.AllenAP"], row["CCF.AllenML"]], as_acronym = True)
    cell_region = scene.atlas.structure_from_coords([row["CCF.AllenDV"], row["CCF.AllenAP"], row["CCF.AllenML"]], as_acronym = True)
    pag_data.at[index, "brainrender.area"] = cell_region
    #pag_data = pag_data.append({"brainrender.area": str(scene.atlas.structure_from_coords([row["CCF.AllenDV"], row["CCF.AllenAP"], row["CCF.AllenML"]], as_acronym = True))}, ignore_index=True)

pag_data.head()
#pag_data.tail()

In [ ]:
you need first to create a list with the data you need, then you can add that as a column of the df
data = []
for index, row in pag_data.iterrows():
    data.append({"brainrender.area": print([scene.atlas.structure_from_coords([row["CCF.AllenDV"], row["CCF.AllenAP"], row["CCF.AllenML"]], as_acronym = True)])}, ignore_index=True)
and then pagadata["brainrender_area"] = data
there might be a more elegant way to do it without the list inbetween, but this is a safe way

#### 4.2 | Using kim_unified_50um_v0.1

In [ ]:
# Scale up data: sharptrack uses a 10um reference atlas, brainrender's default is 1um, and the kim atlas is either 50um or 25um, so the coordinates need to be scaled
brainrender_to_kim_50_scale_factor = 0.02 # the kim atlas we are using is at 50um resolution so the coordinates need to be scaled to match the root

pag_data["CCF.AllenAP"] *= brainrender_to_kim_50_scale_factor
pag_data["CCF.AllenDV"] *= brainrender_to_kim_50_scale_factor
pag_data["CCF.AllenML"] *= brainrender_to_kim_50_scale_factor

pag_data[["cell.id", "cell.type", "CCF.AllenAP", "CCF.AllenDV", "CCF.AllenML"]]